In [1]:
import pandas as pd

df = pd.read_csv("df_ligas_tratado.csv")

df.head()

,fixture.date,fixture.venue.id,fixture.venue.name,fixture.venue.city,fixture.status.short,league.id,league.name,league.country,teams.home.id,teams.home.name,teams.away.id,teams.away.name,goals.home,goals.away,winner,winner.team
0,2023-08-18,755.0,wohninvest WESERSTADION,Bremen,FT,78,Bundesliga,Germany,162,Werder Bremen,157,Bayern Munich,0,4,Vitória_Visitante,Bayern Munich
1,2023-08-19,752.0,Volkswagen Arena,Wolfsburg,FT,78,Bundesliga,Germany,161,VfL Wolfsburg,180,FC Heidenheim,2,0,Vitória_Casa,VfL Wolfsburg
2,2023-08-19,724.0,PreZero Arena,Sinsheim,FT,78,Bundesliga,Germany,167,1899 Hoffenheim,160,SC Freiburg,1,2,Vitória_Visitante,SC Freiburg
3,2023-08-19,699.0,BayArena,Leverkusen,FT,78,Bundesliga,Germany,168,Bayer Leverkusen,173,RB Leipzig,3,2,Vitória_Casa,Bayer Leverkusen
4,2023-08-19,698.0,WWK Arena,Augsburg,FT,78,Bundesliga,Germany,170,FC Augsburg,163,Borussia Monchengladbach,4,4,Empate,-


In [11]:
import plotly.graph_objects as go

partidas_liga = df["league.name"].value_counts()

fig = go.Figure(
    go.Bar(x=partidas_liga.index, y=partidas_liga.values)
)

fig.update_layout(title_text="Partidas por Liga")
fig.show()